In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/drive/MyDrive/ML Projects/Sales_Forecasting_Project/data/raw/sales.csv")
# print(df.head())
# print(df.info())

df["date"] = pd.to_datetime(df["date"])
daily = (
    df.groupby(["date", "sku_id", "category"], as_index=False)
      .agg(daily_qty=("quantity", "sum"))
)
all_dates = pd.date_range(
    start=daily["date"].min(),
    end=daily["date"].max(),
    freq="D"
)

all_skus = daily[["sku_id", "category"]].drop_duplicates()

grid = (
    all_skus.assign(key=1)
    .merge(pd.DataFrame({"date": all_dates, "key": 1}), on="key")
    .drop("key", axis=1)
)
daily = grid.merge(
    daily,
    on=["date", "sku_id", "category"],
    how="left"
)

daily["daily_qty"] = daily["daily_qty"].fillna(0)
daily["day_of_week"] = daily["date"].dt.weekday   # 0=Monday
daily["day_of_year"] = daily["date"].dt.dayofyear
daily["is_weekend"] = (daily["day_of_week"] == 6).astype(int)
# daily["day_of_month"] = daily["date"].dt.day
# daily["month"] = daily["date"].dt.month

daily = daily.sort_values(["sku_id", "date"])
daily["qty_lag_1"] = daily.groupby("sku_id")["daily_qty"].shift(1)
daily["qty_lag_7"] = daily.groupby("sku_id")["daily_qty"].shift(7)
daily["qty_lag_14"] = daily.groupby("sku_id")["daily_qty"].shift(14)

daily["qty_roll_7"] = (
    daily.groupby("sku_id")["daily_qty"]
         .shift(1)
         .rolling(7)
         .mean()
)

daily["qty_roll_14"] = (
    daily.groupby("sku_id")["daily_qty"]
         .shift(1)
         .rolling(14)
         .mean()
)

#for fasting seasons
fasting = pd.read_csv("/content/drive/MyDrive/ML Projects/Sales_Forecasting_Project/data/raw/fasting_periods.csv")
fasting["start_date"] = pd.to_datetime(fasting["start_date"])
fasting["end_date"] = pd.to_datetime(fasting["end_date"])

daily["is_fasting"] = 0

for _, row in fasting.iterrows():
    mask = (daily["date"] >= row["start_date"]) & (daily["date"] <= row["end_date"])
    daily.loc[mask, "is_fasting"] = 1

# for Holiday events
events = pd.read_csv("/content/drive/MyDrive/ML Projects/Sales_Forecasting_Project/data/raw/ethiopian_events.csv")
events["date"] = pd.to_datetime(events["date"])

daily = daily.merge(
    events[["date", "is_holiday"]],
    on="date",
    how="left"
)

daily["is_holiday"] = daily["is_holiday"].fillna(0).astype(int)

holiday_dates = events[events["is_holiday"] == 1]["date"].sort_values()

def days_to_next_holiday(d):
    future = holiday_dates[holiday_dates >= d]
    if len(future) == 0:
        return np.nan
    return (future.iloc[0] - d).days

daily["days_to_holiday"] = daily["date"].apply(days_to_next_holiday)

def days_since_last_holiday(d):
    past = holiday_dates[holiday_dates <= d]
    if len(past) == 0:
        return np.nan
    return (d - past.iloc[-1]).days

daily["days_since_holiday"] = daily["date"].apply(days_since_last_holiday)



daily["target_qty"] = daily.groupby("sku_id")["daily_qty"].shift(-1)

training_df = daily.dropna().reset_index(drop=True)

train_df = training_df[training_df["date"] < "2023-09-11"]
test_df  = training_df[training_df["date"] >= "2023-09-11"]

print(train_df.head())
print(train_df.columns)

train_df.to_csv("/content/drive/MyDrive/ML Projects/Sales_Forecasting_Project/data/processed/train_data.csv", index=False)
test_df.to_csv("/content/drive/MyDrive/ML Projects/Sales_Forecasting_Project/data/processed/test_data.csv", index=False)


       sku_id              category       date  daily_qty  day_of_week  \
0  555-HOU001  Household Essentials 2020-09-25        1.0            4   
1  555-HOU001  Household Essentials 2020-09-26        3.0            5   
2  555-HOU001  Household Essentials 2020-09-27        5.0            6   
3  555-HOU001  Household Essentials 2020-09-28        1.0            0   
4  555-HOU001  Household Essentials 2020-09-29        1.0            1   

   day_of_year  is_weekend  qty_lag_1  qty_lag_7  qty_lag_14  qty_roll_7  \
0          269           0        2.0        5.0         0.0    3.714286   
1          270           0        1.0        4.0         0.0    3.142857   
2          271           1        3.0        2.0         9.0    3.000000   
3          272           0        5.0        3.0         4.0    3.428571   
4          273           0        1.0        5.0         5.0    3.142857   

   qty_roll_14  is_fasting  is_holiday  days_to_holiday  days_since_holiday  \
0     3.285714     

In [ ]:
FEATURES = [
    "day_of_week",
    "is_weekend",
    "day_of_year",
    "qty_lag_1",
    "qty_lag_7",
    "qty_lag_14",
    "qty_roll_7",
    "qty_roll_14",
    "is_holiday",
    "days_to_holiday",
    "days_since_holiday",
    "is_fasting"
]

TARGET = "target_qty"

X_train = train_df[FEATURES]
y_train = train_df[TARGET]

X_test = test_df[FEATURES]
y_test = test_df[TARGET]



NameError: name 'train_df' is not defined

In [ ]:
# training the model
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [ ]:
# making pridiction
y_pred = model.predict(X_test)


In [ ]:
# MAE evaluation
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)


MAE: 3.467496882003968


In [ ]:
# SKU Level MAE to know which product is predictable
test_df["prediction"] = y_pred
test_df["abs_error"] = abs(test_df["prediction"] - test_df["target_qty"])

sku_mae = (
    test_df.groupby("sku_id")["abs_error"]
    .mean()
    .sort_values(ascending=False)
)

print(sku_mae.head(513))


sku_id
SUN-BAL001    12.636981
TOP-DRI004    12.607360
BEE-FRU001    11.379089
BAN-FRU001    11.361319
ድንብ-FRU001    11.350647
                ...    
LAR-HOU003     1.664522
TUM-HOU009     1.662824
HER-CAN002     1.659252
FAR-HOU002     1.653181
LAV-CAN003     1.645107
Name: abs_error, Length: 513, dtype: float64


/tmp/ipython-input-4127590958.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["prediction"] = y_pred
/tmp/ipython-input-4127590958.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["abs_error"] = abs(test_df["prediction"] - test_df["target_qty"])
